<a href="https://colab.research.google.com/github/Ravviii555/Covid_18_Fake_News_Detection/blob/main/fake_news_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
fake = [
    "COVID vaccine is dangerous",
    "Coronavirus is a hoax",
    "Drinking alcohol cures COVID",
    "5G towers spread coronavirus",
    "COVID is fake news"
]
real = [
    "Doctors recommend COVID vaccination",
    "WHO confirms coronavirus transmission",
    "Masks help reduce infection",
    "Vaccines reduce severe illness",
    "Hospitals treat COVID patients"
]

data = fake * 100 + real * 100
labels = [0]*500 + [1]*500  # Regression class

df = pd.DataFrame({"text": data, "label": labels})

print(df.head())
print("Total samples:", len(df))
vectorizer = TfidfVectorizer(stop_words="english", max_features=500)
X = vectorizer.fit_transform(df["text"]).toarray()
y = np.array(df["label"])
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation="relu", input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dense(1)   # Regression output (no sigmoid)
])

model.compile(
    optimizer="adam",
    loss="mse",
    metrics=["mae"]
)

model.summary()
history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=16,
    validation_data=(X_test, y_test)
)
pred = model.predict(X_test)
print("MSE:", mean_squared_error(y_test, pred))
plt.plot(history.history['loss'], label="Train Loss")
plt.plot(history.history['val_loss'], label="Validation Loss")
plt.title("Regression Model Loss Graph")
plt.xlabel("Epochs")
plt.ylabel("Mean Squared Error")
plt.legend()
plt.show()
def test_news(news):
    vector = vectorizer.transform([news]).toarray()
    score = model.predict(vector)[0][0]
    print("Regression Score:", score)

    if score >= 0.5:
        print("Prediction: REAL ✅")
    else:
        print("Prediction: FAKE ❌")
test_news("COVID vaccine causes infertility")

